In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# RUTAS

### SETTLEMENT MASTER LOG

In [ ]:
sml_files = glob.glob(r'G:\My Drive\RESPALDO-PC\DYNAMO\2023\04. ABRIL\SETTLEMENT MASTER LOG\*.tsv', recursive=False)
sml_files

In [ ]:
use_cols_sml = ['MVNT_ID','PSP_AMOUNT','GROSS_AMOUNT','FEES','TAXES','NET_AMOUNT','MDR','MSI','NUMBER_MSI','RECEIPT_NO',
                'STATUS_CODE']

### AP JOURNAL

In [ ]:
ap_files = glob.glob(r'G:\My Drive\RESPALDO-PC\DYNAMO\2023\04. ABRIL\AP JOURNAL\*.tsv', recursive=False)
ap_files

# DATA SETTLEMENT MASTER LOG

In [ ]:
sml = pd.concat([pd.read_csv(f,sep='\t',usecols=use_cols_sml).assign(file_name=os.path.basename(f)) for f in sml_files])
sml.columns = sml.columns.str.strip().str.lower()
sml.replace('""','', regex=True, inplace=True)
# sml['created_at'] = pd.to_datetime(sml['created_at']).dt.tz_localize(None)
# sml['created_at_date'] =  pd.to_datetime(sml['created_at']).dt.date
# sml['created_at_date'] =  pd.to_datetime(sml['created_at_date'])
# sml['eff_dt'] = pd.to_datetime(sml['eff_dt']).dt.tz_localize(None)
# sml['eff_dt_date'] =  pd.to_datetime(sml['eff_dt']).dt.date
# sml['eff_dt_date'] =  pd.to_datetime(sml['eff_dt_date'])
# sml.rename(columns={'ref_id':'transaction_id'},inplace=True)
#sml['PK'] = sml['PK'].replace('M#','',inplace=True)
sml.head()

In [ ]:
sml.shape

In [ ]:
sml.dtypes

In [ ]:
sml_final = sml
sml_final.head()

In [ ]:
sml_final.shape

# DATA AP JOURNAL

In [ ]:
ap = pd.concat([pd.read_csv(f, sep='\t').assign(file_name=os.path.basename(f)) for f in ap_files])
ap.columns = ap.columns.str.strip().str.lower()
ap.replace('""','', regex=True, inplace=True)
ap['eff_dt'] =  pd.to_datetime(ap['eff_dt']).dt.tz_localize(None)
ap['eff_dt_date'] =  pd.to_datetime(ap['eff_dt']).dt.date
ap['eff_dt_date'] =  pd.to_datetime(ap['eff_dt_date'])
ap['eff_dt_time'] =  pd.to_datetime(ap['eff_dt']).dt.time
ap['movement_day'] = pd.to_datetime(ap['movement_day']).dt.tz_localize(None)
ap['created_at'] = pd.to_datetime(ap['created_at']).dt.tz_localize(None)
ap['created_at_date'] =  pd.to_datetime(ap['created_at']).dt.date
ap['created_at_date'] =  pd.to_datetime(ap['created_at_date'])
ap['request_effective_date'] = pd.to_datetime(ap['request_effective_date']).dt.tz_localize(None)
ap['slot_date'] = pd.to_datetime(ap['slot_date']).dt.tz_localize(None)
ap['identifier'] = ap['source_type'].str[:1].astype(str)
ap['INDEX'] = (ap['eff_dt_date'].astype(str).replace(' ', '',regex=True)+'-'+
               ap['identifier'].astype(str).replace(' ', '',regex=True)+'-'+
               ap['eff_dt_time'].astype(str).replace(' ', '',regex=True))
ap.sort_values(by=(['merchant_id','eff_dt']), ascending=[True,False], inplace=True)

In [ ]:
ap.reset_index(inplace=True)

In [ ]:
ap.loc[ap['source_type'] == 'DISB', 'report_id'] = ap['type_request']
ap['report_id'] = ap['report_id'].groupby(ap['merchant_id']).fillna(method='ffill')
ap.head()

In [ ]:
ap.shape

In [ ]:
ap.dtypes

In [ ]:
ap['source_type'].unique()

In [ ]:
ap['report_id'].unique()

In [ ]:
ap_sett = ap[(ap['source_type'] == 'SETT')]
ap_topups = ap[(ap['source_type'] == 'TOPUPS')]
ap_disb = ap[(ap['source_type'] == 'DISB')]
ap_loan = ap[(ap['source_type'] == 'LOAN')]
ap_pymt = ap[(ap['source_type'] == 'PYMT')]
ap_risk = ap[(ap['source_type'] == 'RISK')]
ap_sett_adptr = ap[(ap['source_type'] == 'SETT_ADPTR')]
ap_taxes = ap[(ap['source_type'] == 'TAXES')]
ap_refunds = ap[(ap['source_type'] == 'REFUNDS')]
ap_bill_pymnt = ap[(ap['source_type'] == 'BILL_PYMNT')]

# CRUCE 1: AP JOURNAL VS SETTLEMENT MASTER LOG

In [ ]:
cruce_1 = ap.merge(sml_final[['mvnt_id','psp_amount','gross_amount','fees','taxes','net_amount','mdr','msi','number_msi',
                             'receipt_no','status_code']],on='mvnt_id',how='left')
cruce_1['transaction_id'] = cruce_1['mvnt_id']
cruce_1.head()

In [ ]:
cruce_1.shape

In [ ]:
cruce_1.columns

# DATA PROVISION

In [ ]:
provision = pd.read_excel(r'C:\Users\Cristian.Aparicio\Documents\PROVISIONES\2023\04. ABRIL\Provision Abril 2023 vf.xlsx',
                          sheet_name='ECOMMERCE',
                          converters={'AFFILIATION':str,'SUB_AFFILIATION':str,'ORDER_ID':str,'COD_AUTORIZACION':str,
                                     'BIN':str,'ULTIMOS_4':str})
provision.columns = provision.columns.str.strip().str.lower() 
provision.head()

In [ ]:
provision.shape

# CRUCE 2: DATA PROVISION VS CRUCE 1

In [ ]:
cruce_2 = provision.merge(cruce_1[['transaction_id','mvnt_id','report_id','fees','taxes','eff_dt_date','movement_type',
                                  'source_type']],on='transaction_id',how='left')
cruce_2['report_id'].fillna('not_found',inplace=True)
cruce_2['movement_type'].fillna('not_found',inplace=True)
cruce_2['source_type'].fillna('not_found',inplace=True)
cruce_2.head()

In [ ]:
cruce_2.shape

In [ ]:
cruce_2['movement_type'].unique()

In [ ]:
cruce_2['source_type'].unique()

In [ ]:
cruce_2.to_excel('ECOMMERCE_ABRIL_2023.xlsx',index=False)

###### CON EL ARCHIVO GENERADO BUSCAR LOS REPORT_ID EN EL ESCRITORIO REMOTO

# DATA PAGOS

In [ ]:
pagos = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\PROVISIONES\2023\04. ABRIL\0. DATA PAGOS\Pagos_grrcn_abril_2023.csv')
pagos.columns = pagos.columns.str.strip().str.lower() 
pagos['report_date_local'] =  pd.to_datetime(pagos['report_date_local'])
pagos['payment_date_local'] =  pd.to_datetime(pagos['payment_date_local'])
pagos['batch_created_date'] =  pd.to_datetime(pagos['batch_created_date'])
pagos.sort_values(by=(['report_id','batch_created_date']), ascending=[True,False], inplace=True)
pagos.head()

In [ ]:
pagos.shape

In [ ]:
pagos_duplicates = pagos[pagos.duplicated(['report_id'] , keep=False)]
pagos_duplicates.head()

In [ ]:
pagos_duplicates.shape

In [ ]:
pagos.drop_duplicates(subset='report_id', keep='first', inplace=True) 

In [ ]:
pagos.head()

In [ ]:
pagos.shape

# CRUCE 3: DATA PAGOS VS CRUCE 2

In [ ]:
cruce_3 = cruce_2.merge(pagos[['report_id','report_date_local','payment_date_local','batch_created_date','status']],
                        on='report_id',how='left')
cruce_3['status'].fillna('not_found',inplace=True)
cruce_3.head()

In [ ]:
cruce_3.shape

In [ ]:
cruce_3.to_excel('GRRCN_ABRIL_2023_FINAL.xlsx',index=False)